In [22]:
import pandas as pd
from music21 import converter, instrument, note, chord
from sklearn.preprocessing import LabelEncoder
import itertools
import warnings
warnings.filterwarnings("ignore")

In [23]:
path = 'data\maestro-v3.0.0.csv'
df = pd.read_csv(path)

In [24]:
#df = df.sort_values(by='duration').iloc[:50]
#df = df[df['canonical_composer'].isin(df['canonical_composer'].unique()[:10])]
#compositeur avec le plus de track 
comp = df.groupby('canonical_composer')['midi_filename'].count().reset_index().sort_values(by='midi_filename')['canonical_composer'].to_list()[-5:-4]
df = df[df['canonical_composer'].isin(comp)]

In [26]:
Lnom,Lmidi = [],[]
def extraction_notes(file):
    midi = converter.parse(file)
    L_notes = []
    for e in midi.recurse().notes:
        Lmidi.append(e)
        if e.isNote:
            L_notes.append(note.Note(e.pitch.midi).nameWithOctave)
            Lnom.append(note.Note(e.pitch.midi).nameWithOctave)
        elif e.isChord:
            ch = chord.Chord(e)
            if ch.commonName:
                L_notes.append(ch.commonName) 
                Lnom.append(ch.commonName) 
            elif ch.figure:
                L_notes.append(ch.figure)
                Lnom.append(ch.figure)
            else:
                L_notes.append(ch.pitchedCommonName) 
                Lnom.append(ch.pitchedCommonName) 
    if len(L_notes)<200:
        return 'null'
    else:
        return L_notes 
    
def ex_notes_tab(df,col_out):
    df['midi_file_data'] = 'data/'+df['midi_filename']
    df[col_out] = df['midi_file_data'].apply(extraction_notes)
    return df[df[col_out]!='null']

In [27]:
def create_map(dftrain,dftest,deval,col):
    all_notes = notes_df(dftrain) + notes_df(dftest) + notes_df(deval)
    all_notes = list(set(all_notes))
    global Dmap

    Dmap={}
    i=0
    for k in all_notes:
        Dmap[k]=i
        i+=1

    dftrain[col] = dftrain[col].apply(map_liste)
    dftest[col] = dftest[col].apply(map_liste)
    deval[col] = deval[col].apply(map_liste)
    
    return dftrain,dftest,deval

def notes_df(df,col='notes'):
    all_notes = []
    for x in df[col].to_list():
        all_notes+=x
        all_notes = list(set(all_notes))
    return list(set(all_notes))

def map_liste(l):
    return [Dmap[e] for e in l]

In [28]:
def colonnes(df):
    df_expanded = df['X'].apply(pd.Series)
    df_expanded['y'] = df['y']

    return df_expanded

def sequence_list(L):
    sequence_length = 200

    input_sequences = []
    output_notes = []

    for i in range(len(L) - sequence_length):
        seq_in = L[i:i + sequence_length]  
        seq_out = L[i + sequence_length]  
        input_sequences.append(seq_in)
        output_notes.append(seq_out)

    return input_sequences,output_notes

def sequence(dff):
    X,y = [],[]
    for k in dff['notes'].to_list():
        xtemp,ytemp = sequence_list(k)
        X.append(xtemp)
        y.append(ytemp)
    X=list(itertools.chain(*X))
    y=list(itertools.chain(*y))

    return pd.DataFrame({"X":X,"y":y})

In [29]:
def preprocessing(df):

    dftrain = ex_notes_tab(df[df['split']=='train'],'notes')
    dftest = ex_notes_tab(df[df['split']=='test'],'notes')
    deval = ex_notes_tab(df[df['split']=='validation'],'notes')

    dftrain,dftest,deval = create_map(dftrain,dftest,deval,'notes')

    return colonnes(sequence(dftrain)),colonnes(sequence(dftest)),colonnes(sequence(deval))
    

In [30]:
dftrain,dftest,deval = preprocessing(df)

KeyboardInterrupt: 

In [ ]:
dftrain.to_csv('00_train_'+comp[0]+'50_V8'.csv',index=False)
dftest.to_csv('00_test_50_V8.csv',index=False)
deval.to_csv('00_validation_50_V8.csv',index=False)

# EXPORTTTTTT

In [37]:
nomtomidi = {}
for k in range(len(Lmidi)):
    nomtomidi[Lnom[k]]=[Lmidi[k]]


In [38]:
encodetomidi = {}
for k in nomtomidi:
    encodetomidi[Dmap[k]]=nomtomidi[k]

In [39]:
generated = [137, 245, 230, 130, 205, 46, 100, 245, 32, 253, 350, 67, 108, 245, 248, 37, 346, 245, 165, 65, 152, 113, 115, 130, 65, 23, 113, 248, 16, 197, 341, 245, 201, 107, 107, 248, 46, 23, 46, 32, 371, 69, 100, 46, 245, 201, 25, 245, 113, 46, 32, 27, 350, 46, 251, 245, 300, 245, 100, 16, 234, 43, 346, 245, 201, 201, 201, 201, 201, 201, 201, 201, 201, 201, 201, 245, 46, 255, 46, 245, 245, 255, 21, 133, 201, 46, 346, 245, 147, 245, 46, 23, 152, 72, 245, 201, 245, 46, 23, 245]

In [40]:
generated_midi = []
for k in generated:
    generated_midi.append(encodetomidi[k])

In [41]:
from music21 import stream
from copy import deepcopy
s = stream.Stream()

# Paramètres : durée de chaque note ou accord
duration = 0.5  # durée en quarts de note

# Ajoute chaque note ou accord à la Stream
for group in generated_midi:
    for element in group:
        element_copy = deepcopy(element)
        element_copy.quarterLength = duration
        s.append(element_copy)

# Exporte en MIDI
s.write('midi', fp='test_v2_02.mid')

'test_v2_02.mid'